In [ ]:
import gym
from collections import namedtuple
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as layers


In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [ ]:

class Net(tf.keras.Model):
    def get_config(self):
        pass
    def __init__(self, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = tf.keras.Sequential([
            layers.Dense(hidden_size),
            layers.ReLU(),
            layers.Dense(n_actions)
        ])

    def call(self, inputs, training=None, mask=None):
        return self.net(inputs)


In [ ]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [ ]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple("EpisodeStep", field_names=['observation', 'action'])

In [ ]:

def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    softmax = layers.Softmax(axis=1)
    while True:
        obs_v = tf.convert_to_tensor([obs])
        act_probs_v = softmax(net(obs_v))
        act_probs = act_probs_v.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


In [ ]:

def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = tf.convert_to_tensor(train_obs)
    train_act_v = tf.convert_to_tensor(train_act, dtype=tf.int64)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [ ]:

env = gym.make("FrozenLake-v0")
env = DiscreteOneHotWrapper(env)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(HIDDEN_SIZE, n_actions)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

log_dir = "logs/cartpole_cross_entropy"
writer = tf.summary.create_file_writer(logdir=log_dir)

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    with tf.GradientTape() as g:
        action_scores_v = net(obs_v)
        loss_v = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(acts_v, action_scores_v))

    gradients = g.gradient(loss_v, sources=net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))

    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.numpy(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v, iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)

    if reward_m > .8:
        print("Solved!")
        break


